In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
mnist = fetch_openml('mnist_784')

In [5]:
data, target = mnist["data"], mnist["target"]

print(f"data의 type   : {type(data)},  data의 shape : {data.shape}")
print(f"target의 type : {type(target)},    target의 shape : {target.shape}")

data의 type   : <class 'pandas.core.frame.DataFrame'>,  data의 shape : (70000, 784)
target의 type : <class 'pandas.core.series.Series'>,    target의 shape : (70000,)


In [6]:
target.dtypes

CategoricalDtype(categories=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], ordered=False)

In [7]:
target = target.astype(np.int8)
data, target = data.values, target.values

print(f"data의 type   : {type(data)},  data의 shape : {data.shape}")
print(f"target의 type : {type(target)},    target의 shape : {target.shape}")

data의 type   : <class 'numpy.ndarray'>,  data의 shape : (70000, 784)
target의 type : <class 'numpy.ndarray'>,    target의 shape : (70000,)


In [8]:
print(target[10:20])

[3 5 3 6 1 7 2 8 6 9]


In [9]:
target_dict = {}
label, freq = np.unique(target, return_counts=True)

for l, f in zip(label, freq):
    target_dict[l] = f
    
print(target_dict)

{0: 6903, 1: 7877, 2: 6990, 3: 7141, 4: 6824, 5: 6313, 6: 6876, 7: 7293, 8: 6825, 9: 6958}


In [10]:
print(np.max(data), np.min(data))

data = data / 255      # To keep our gradients manageable
print(np.max(data), np.min(data))

255.0 0.0
1.0 0.0


In [13]:
digits = 10
examples = target.shape[0]     # 70000
target_new = np.eye(digits)[target.reshape(1, -1)]

In [15]:
print(f"target_new의 shape : {target_new.shape}")
print()
print(target[10:20])
print()
print(target_new[0, 10:20, :])

target_new의 shape : (1, 70000, 10)

[3 5 3 6 1 7 2 8 6 9]

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [17]:
target_new = target_new.reshape(-1, 10)
print(f"target의 shape : {target_new.shape}")
print()
print(target_new[10:20, :])

target의 shape : (70000, 10)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
